# Melbourne Housing Project

Source Data: https://discover.data.vic.gov.au/dataset/rental-report-quarterly-quarterly-median-rents-by-lga<br>
<br>Acessed: 19/01/2026

In [1]:
# Libraries
import pandas as pd
import geopandas as gpd
import os
import regex as re

## Cleaning the data

In [2]:
# Read the raw data, since there might be more than one header
data_path =r"..\data\raw\Quarterly_median_rents_JuneQuarter2025.xlsx"

# Checking if there are +1 sheet
excel_file = pd.ExcelFile(data_path)
print("Available sheets:")
print(excel_file.sheet_names)

Available sheets:
['1br flat', '2br Flat', '3br Flat', '2br House', '3br House', '4br House', 'All Properties']


In [3]:
# Loading only the sheet I currently have interest in
# 1 bedroom flat
df_1br_flat = pd.read_excel(data_path, sheet_name='1br flat', header=1)
df_1br_flat.head(10)

,1 bedroom flats,Unnamed: 1,Jun 1999,Jun 1999.1,Sep 1999,Sep 1999.1,Dec 1999,Dec 1999.1,Mar 2000,Mar 2000.1,...,Jun 2024,Jun 2024.1,Sep 2024,Sep 2024.1,Dec 2024,Dec 2024.1,Mar 2025,Mar 2025.1,Jun 2025,Jun 2025.1
0,NaN,NaN,Count,Median,Count,Median,Count,Median,Count,Median,...,Count,Median,Count,Median,Count,Median,Count,Median,Count,Median
1,Barwon South West,Colac-Otway,6,68,6,75,7,80,6,78,...,7,300,9,290,-,-,-,-,5,254
2,NaN,Corangamite,-,-,5,60,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,NaN,Glenelg,7,65,-,-,13,70,9,80,...,7,235,5,250,-,-,7,340,-,-
4,NaN,Greater Geelong,202,80,204,80,198,80,254,85,...,139,330,163,325,148,330,150,340,115,350
5,NaN,Moyne,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
6,NaN,Queenscliffe,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
7,NaN,Southern Grampians,9,65,10,68,7,70,10,68,...,10,240,7,240,-,-,6,260,9,250
8,NaN,Surf Coast,9,100,9,92,9,100,8,118,...,7,360,6,293,5,236,5,375,-,-
9,NaN,Warrnambool,28,75,23,75,26,75,36,78,...,6,315,15,300,13,325,6,386,14,355


In [4]:
# Ok, now let's copy this data on a new df excluding the first header line
hprice_1br = df_1br_flat.copy()

In [5]:
# And lets drop the first column, with the region name, since we have all the government areas named on the 2nd column
hprice_1br.drop(columns=hprice_1br.columns[0])

,Unnamed: 1,Jun 1999,Jun 1999.1,Sep 1999,Sep 1999.1,Dec 1999,Dec 1999.1,Mar 2000,Mar 2000.1,Jun 2000,...,Jun 2024,Jun 2024.1,Sep 2024,Sep 2024.1,Dec 2024,Dec 2024.1,Mar 2025,Mar 2025.1,Jun 2025,Jun 2025.1
0,NaN,Count,Median,Count,Median,Count,Median,Count,Median,Count,...,Count,Median,Count,Median,Count,Median,Count,Median,Count,Median
1,Colac-Otway,6,68,6,75,7,80,6,78,11,...,7,300,9,290,-,-,-,-,5,254
2,Corangamite,-,-,5,60,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,Glenelg,7,65,-,-,13,70,9,80,8,...,7,235,5,250,-,-,7,340,-,-
4,Greater Geelong,202,80,204,80,198,80,254,85,187,...,139,330,163,325,148,330,150,340,115,350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,Metro,3618,130,3902,135,3747,135,4547,138,3907,...,7297,470,8280,480,7107,470,9174,500,6564,495
91,Non-Metro,759,80,685,80,726,80,923,80,706,...,557,280,564,280,515,290,519,300,531,300


In [6]:
print("Raw shape:", hprice_1br.shape)
print("\nColumn names (first 10):")
print(hprice_1br.columns[:10].tolist())
print("\nColumn names (last 10):")
print(hprice_1br.columns[-10:].tolist())

# Get the Count/Median row (currently row 0 in the data)
print("\nFirst data row (Count/Median labels):")
print(hprice_1br.iloc[0])

Raw shape: (93, 212)

Column names (first 10):
['1 bedroom flats', 'Unnamed: 1', 'Jun 1999', 'Jun 1999.1', 'Sep 1999', 'Sep 1999.1', 'Dec 1999', 'Dec 1999.1', 'Mar 2000', 'Mar 2000.1']

Column names (last 10):
['Jun 2024', 'Jun 2024.1', 'Sep 2024', 'Sep 2024.1', 'Dec 2024', 'Dec 2024.1', 'Mar 2025', 'Mar 2025.1', 'Jun 2025', 'Jun 2025.1']

First data row (Count/Median labels):
1 bedroom flats       NaN
Unnamed: 1            NaN
Jun 1999            Count
Jun 1999.1         Median
Sep 1999            Count
                    ...  
Dec 2024.1         Median
Mar 2025            Count
Mar 2025.1         Median
Jun 2025            Count
Jun 2025.1         Median
Name: 0, Length: 212, dtype: object


In [7]:
# Creating new column names
new_columns = ['Region']

for col in hprice_1br.columns[1:]:  # Skip first column (Unnamed: 1)
    # Extract quarter (remove .1, .2 suffixes from duplicates)
    quarter = col.split('.')[0] if '.' in col else col
    
    # Get Count or Median from first row
    count_or_median = hprice_1br[col].iloc[0]
    
    # Create column name: Jun_1999_Count, Jun_1999_Median, etc.
    new_col_name = f"{quarter}_{count_or_median}".replace(' ', '_')
    new_columns.append(new_col_name)

print(f"\nNew columns (first 10): {new_columns[:10]}")
print(f"New columns (last 10): {new_columns[-10:]}")


New columns (first 10): ['Region', 'Unnamed:_1_nan', 'Jun_1999_Count', 'Jun_1999_Median', 'Sep_1999_Count', 'Sep_1999_Median', 'Dec_1999_Count', 'Dec_1999_Median', 'Mar_2000_Count', 'Mar_2000_Median']
New columns (last 10): ['Jun_2024_Count', 'Jun_2024_Median', 'Sep_2024_Count', 'Sep_2024_Median', 'Dec_2024_Count', 'Dec_2024_Median', 'Mar_2025_Count', 'Mar_2025_Median', 'Jun_2025_Count', 'Jun_2025_Median']


In [8]:
# Create proper column names
new_columns = []

for i, col in enumerate(hprice_1br.columns):
    if i == 0:
        # First column is the property type title - skip it
        continue
    elif i == 1:
        # Second column (Unnamed: 1) contains regions
        new_columns.append('Region')
    else:
        # Extract quarter (remove .1 suffix from duplicates)
        quarter = col.split('.')[0]
        
        # Get Count or Median from first row
        count_or_median = hprice_1br[col].iloc[0]
        
        # Create column name: Jun_1999_Count, Jun_1999_Median, etc.
        new_col_name = f"{quarter}_{count_or_median}".replace(' ', '_')
        new_columns.append(new_col_name)

# Apply new column names (skip first column with title)
hprice_1br_clean = hprice_1br.iloc[:, 1:].copy()
hprice_1br_clean.columns = new_columns

In [9]:
# Remove the first row (Count/Median labels row)
hprice_1br_clean = hprice_1br_clean.iloc[1:].reset_index(drop=True)

In [10]:
# Convert numeric columns to proper types (replace '--' with NaN)
hprice_1br_clean = hprice_1br_clean.replace('--', pd.NA)

print("Cleaned shape:", hprice_1br_clean.shape)
print("\nFirst 10 rows:")
print(hprice_1br_clean.head(10))
print("\nColumn names (first 10):")
print(hprice_1br_clean.columns[:10].tolist())

Cleaned shape: (92, 211)

First 10 rows:
               Region Jun_1999_Count Jun_1999_Median Sep_1999_Count  \
0         Colac-Otway              6              68              6   
1         Corangamite              -               -              5   
2             Glenelg              7              65              -   
3     Greater Geelong            202              80            204   
4               Moyne              -               -              -   
5        Queenscliffe              -               -              -   
6  Southern Grampians              9              65             10   
7          Surf Coast              9             100              9   
8         Warrnambool             28              75             23   
9         Group Total            264              80            261   

  Sep_1999_Median Dec_1999_Count Dec_1999_Median Mar_2000_Count  \
0              75              7              80              6   
1              60              -           

In [11]:
# Checking the data types
print("\nData types:")
print(hprice_1br_clean.dtypes.head(10))


Data types:
Region                str
Jun_1999_Count     object
Jun_1999_Median    object
Sep_1999_Count     object
Sep_1999_Median    object
Dec_1999_Count     object
Dec_1999_Median    object
Mar_2000_Count     object
Mar_2000_Median    object
Jun_2000_Count     object
dtype: object


In [12]:
# Now, before converting the data types, let's remove the final spreadsheet rows with summaries
# Checking first
hprice_1br_clean.tail(10)

,Region,Jun_1999_Count,Jun_1999_Median,Sep_1999_Count,Sep_1999_Median,Dec_1999_Count,Dec_1999_Median,Mar_2000_Count,Mar_2000_Median,Jun_2000_Count,...,Jun_2024_Count,Jun_2024_Median,Sep_2024_Count,Sep_2024_Median,Dec_2024_Count,Dec_2024_Median,Mar_2025_Count,Mar_2025_Median,Jun_2025_Count,Jun_2025_Median
82,Kingston,122,110,105,110,82,110,103,115,114,...,103,420,85,410,100,425,99,420,85,430
83,Mornington Penin'a,18,90,26,98,29,100,18,95,15,...,29,350,8,415,19,360,19,370,20,370
84,Port Phillip,689,155,808,160,726,160,825,160,739,...,645,460,682,470,607,480,652,480,615,485
85,Stonnington,429,155,446,155,444,155,540,161,460,...,692,475,828,465,708,452,739,480,594,500
86,Group Total,1755,140,1921,140,1807,145,2050,145,1881,...,2195,450,2277,450,2051,450,2296,450,1900,460
87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,Metro,3618,130,3902,135,3747,135,4547,138,3907,...,7297,470,8280,480,7107,470,9174,500,6564,495
90,Non-Metro,759,80,685,80,726,80,923,80,706,...,557,280,564,280,515,290,519,300,531,300
91,Victoria,4377,120,4587,125,4473,130,5470,130,4613,...,7854,450,8844,470,7622,460,9693,500,7095,480


In [13]:
#So we have the final 5 rows with non-numeric or summary values we are not currently interested in
hprice_1br_clean = hprice_1br_clean.iloc[:-5]

In [14]:
hprice_1br_clean.tail(5)

,Region,Jun_1999_Count,Jun_1999_Median,Sep_1999_Count,Sep_1999_Median,Dec_1999_Count,Dec_1999_Median,Mar_2000_Count,Mar_2000_Median,Jun_2000_Count,...,Jun_2024_Count,Jun_2024_Median,Sep_2024_Count,Sep_2024_Median,Dec_2024_Count,Dec_2024_Median,Mar_2025_Count,Mar_2025_Median,Jun_2025_Count,Jun_2025_Median
82,Kingston,122,110,105,110,82,110,103,115,114,...,103,420,85,410,100,425,99,420,85,430
83,Mornington Penin'a,18,90,26,98,29,100,18,95,15,...,29,350,8,415,19,360,19,370,20,370
84,Port Phillip,689,155,808,160,726,160,825,160,739,...,645,460,682,470,607,480,652,480,615,485
85,Stonnington,429,155,446,155,444,155,540,161,460,...,692,475,828,465,708,452,739,480,594,500
86,Group Total,1755,140,1921,140,1807,145,2050,145,1881,...,2195,450,2277,450,2051,450,2296,450,1900,460


In [15]:
#Shape with the "Group Total" rows
hprice_1br_clean.shape

(87, 211)

In [16]:
# Check how many 'Group Total' rows we have
group_total_mask = hprice_1br_clean['Region'].str.contains('Group Total', case=False, na=False)
print(f"Number of 'Group Total' rows found: {group_total_mask.sum()}")

print("\nGroup Total rows:")
print(hprice_1br_clean[group_total_mask][['Region', 'Jun_1999_Count', 'Jun_1999_Median']])

# Remove Group Total rows
hprice_1br_clean = hprice_1br_clean[~group_total_mask].reset_index(drop=True)

print(f"\nShape after removing Group Total rows: {hprice_1br_clean.shape}")

Number of 'Group Total' rows found: 8

Group Total rows:
         Region Jun_1999_Count Jun_1999_Median
9   Group Total            264              80
21  Group Total            114              80
32  Group Total            130              80
45  Group Total            128              85
52  Group Total            123              70
67  Group Total           1555             120
75  Group Total            308             128
86  Group Total           1755             140

Shape after removing Group Total rows: (79, 211)


In [17]:
# Convert all columns except Region to float
numeric_cols = hprice_1br_clean.columns[1:]
hprice_1br_clean[numeric_cols] = hprice_1br_clean[numeric_cols].apply(pd.to_numeric, errors='coerce')

print("Data types after conversion:")
print(hprice_1br_clean.dtypes.head(10))

print("\nFirst 10 rows after conversion:")
print(hprice_1br_clean.head(10))

print(f"\nShape: {hprice_1br_clean.shape}")

Data types after conversion:
Region                 str
Jun_1999_Count     float64
Jun_1999_Median    float64
Sep_1999_Count     float64
Sep_1999_Median    float64
Dec_1999_Count     float64
Dec_1999_Median    float64
Mar_2000_Count     float64
Mar_2000_Median    float64
Jun_2000_Count     float64
dtype: object

First 10 rows after conversion:
               Region  Jun_1999_Count  Jun_1999_Median  Sep_1999_Count  \
0         Colac-Otway             6.0             68.0             6.0   
1         Corangamite             NaN              NaN             5.0   
2             Glenelg             7.0             65.0             NaN   
3     Greater Geelong           202.0             80.0           204.0   
4               Moyne             NaN              NaN             NaN   
5        Queenscliffe             NaN              NaN             NaN   
6  Southern Grampians             9.0             65.0            10.0   
7          Surf Coast             9.0            100.0        

In [18]:
# Check for any NaN values
print("\nNaN count per column (first 10 columns):")
print(hprice_1br_clean.iloc[:, :10].isna().sum())


NaN count per column (first 10 columns):
Region              0
Jun_1999_Count     24
Jun_1999_Median    24
Sep_1999_Count     26
Sep_1999_Median    26
Dec_1999_Count     26
Dec_1999_Median    26
Mar_2000_Count     25
Mar_2000_Median    25
Jun_2000_Count     22
dtype: int64


In this case, we are unsure if NaN means 0, so we'll keep the NaN records. NaN could mean the number is not representative (smaller than a certain threshold we are unaware) or the region didn't exist at that time. So let us leave as it is and work with these nurmbers.

In [19]:
#Also, for analysis purposes, let separate the data in 2 dfs: count and median

# Get all Count columns
count_cols = ['Region'] + [col for col in hprice_1br_clean.columns if 'Count' in col]
hprice_1br_count = hprice_1br_clean[count_cols].copy()

print("Count DataFrame:")
print(f"Shape: {hprice_1br_count.shape}")
print(hprice_1br_count.head())

Count DataFrame:
Shape: (79, 106)
            Region  Jun_1999_Count  Sep_1999_Count  Dec_1999_Count  \
0      Colac-Otway             6.0             6.0             7.0   
1      Corangamite             NaN             5.0             NaN   
2          Glenelg             7.0             NaN            13.0   
3  Greater Geelong           202.0           204.0           198.0   
4            Moyne             NaN             NaN             NaN   

   Mar_2000_Count  Jun_2000_Count  Sep_2000_Count  Dec_2000_Count  \
0             6.0            11.0            10.0             6.0   
1             NaN             NaN             7.0             NaN   
2             9.0             8.0             9.0             9.0   
3           254.0           187.0           186.0           208.0   
4             NaN             NaN             NaN             NaN   

   Mar_2001_Count  Jun_2001_Count  ...  Mar_2023_Count  Jun_2023_Count  \
0             9.0             6.0  ...            10.0  

In [20]:
# Same for Median

# Get all Median columns
median_cols = ['Region'] + [col for col in hprice_1br_clean.columns if 'Median' in col]
hprice_1br_median = hprice_1br_clean[median_cols].copy()

print("Median DataFrame:")
print(f"Shape: {hprice_1br_median.shape}")
print(hprice_1br_median.head())

Median DataFrame:
Shape: (79, 106)
            Region  Jun_1999_Median  Sep_1999_Median  Dec_1999_Median  \
0      Colac-Otway             68.0             75.0             80.0   
1      Corangamite              NaN             60.0              NaN   
2          Glenelg             65.0              NaN             70.0   
3  Greater Geelong             80.0             80.0             80.0   
4            Moyne              NaN              NaN              NaN   

   Mar_2000_Median  Jun_2000_Median  Sep_2000_Median  Dec_2000_Median  \
0             78.0             75.0             80.0             83.0   
1              NaN              NaN             55.0              NaN   
2             80.0             72.0             70.0             75.0   
3             85.0             85.0             85.0             85.0   
4              NaN              NaN              NaN              NaN   

   Mar_2001_Median  Jun_2001_Median  ...  Mar_2023_Median  Jun_2023_Median  \
0        

In [21]:
# Now lets rename the columns to remove '_Count' and '_Median' suffixes for cleaner column names
hprice_1br_count.columns = ['Region'] + [col.replace('_Count', '') for col in hprice_1br_count.columns[1:]]
hprice_1br_median.columns = ['Region'] + [col.replace('_Median', '') for col in hprice_1br_median.columns[1:]]

print("After renaming - Count columns (first 10):")
print(hprice_1br_count.columns[:10].tolist())

print("\nAfter renaming - Median columns (first 10):")
print(hprice_1br_median.columns[:10].tolist())

After renaming - Count columns (first 10):
['Region', 'Jun_1999', 'Sep_1999', 'Dec_1999', 'Mar_2000', 'Jun_2000', 'Sep_2000', 'Dec_2000', 'Mar_2001', 'Jun_2001']

After renaming - Median columns (first 10):
['Region', 'Jun_1999', 'Sep_1999', 'Dec_1999', 'Mar_2000', 'Jun_2000', 'Sep_2000', 'Dec_2000', 'Mar_2001', 'Jun_2001']


In [23]:
# Also, let's save these new dfs into csv files, so we store the cleaned data
hprice_1br_median_csv = hprice_1br_median.to_csv(r'..\data\processed\hprice_1br_median.csv')
hprice_1br_count = hprice_1br_count.to_csv(r'..\data\processed\hprice_1br_count.csv')

## EDA

First, let's dive into the Median values.

In [22]:
# Ok, so let's get some basic statistics
print("\nBasic statistics for first few numeric columns:")
print(hprice_1br_median.iloc[:, 1:7].describe())


Basic statistics for first few numeric columns:
         Jun_1999    Sep_1999    Dec_1999    Mar_2000    Jun_2000    Sep_2000
count   55.000000   53.000000   53.000000   54.000000   57.000000   58.000000
mean    97.600000  101.075472  101.622642  101.722222  102.210526  102.827586
std     25.958371   29.196845   28.411960   28.531622   30.597465   30.928972
min     65.000000   60.000000   65.000000   65.000000   60.000000   55.000000
25%     79.000000   80.000000   80.000000   80.000000   80.000000   80.750000
50%     90.000000   95.000000   95.000000   92.500000   95.000000   92.500000
75%    115.000000  120.000000  115.000000  119.500000  120.000000  120.000000
max    180.000000  195.000000  200.000000  200.000000  215.000000  220.000000


In [23]:
# Now, let's see this data in charts
# Importing libs
# Remember to run: pip freeze > requirements.txt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# OVERALL TREND: Sum of all median values over time

# Calculating total median rent across all regions for each quarter
median_totals = hprice_1br_median.iloc[:, 1:].sum(axis=0)

# Extracting quarters from column names for x-axis
quarters = hprice_1br_median.columns[1:].tolist()

# Creating a DataFrame for plotting
trend_hprice_1br = pd.DataFrame({
    'Quarter': quarters,
    'Total_Median_Rent': median_totals.values
})

# Creating the plot
fig1 = go.Figure()
fig1.add_trace(go.Scatter(
    x=trend_hprice_1br['Quarter'],
    y=trend_hprice_1br['Total_Median_Rent'],
    mode='lines+markers',
    name='Total Median Rent',
    line=dict(color='royalblue', width=2),
    marker=dict(size=4)
))

fig1.update_layout(
    title='1 Bedroom Flat: Total Median Rent Trend (1999-2025)',
    xaxis_title='Quarter',
    yaxis_title='Sum of Median Rent (AUD)',
    hovermode='x unified',
    height=500,
    xaxis=dict(tickangle=-45)
)

fig1.show()

print(f"Number of quarters analyzed: {len(quarters)}")
print(f"Starting total (Jun 1999): ${median_totals.iloc[0]:,.0f}")
print(f"Ending total (Jun 2025): ${median_totals.iloc[-1]:,.0f}")
print(f"Overall increase: ${median_totals.iloc[-1] - median_totals.iloc[0]:,.0f}")

Number of quarters analyzed: 105
Starting total (Jun 1999): $5,368
Ending total (Jun 2025): $19,411
Overall increase: $14,043


In [28]:
# Now lets calculate the region with the highest median value (across all time periods)

# Calculate mean median rent for each region
region_avg_median = hprice_1br_median.set_index('Region').mean(axis=1, skipna=True)

highest_region = region_avg_median.idxmax()
highest_value = region_avg_median.max()

print(f"HIGHEST AVERAGE MEDIAN RENT:")
print(f"Region: {highest_region}")
print(f"Average Median Rent: ${highest_value:,.2f}")

# Show trend for this region
highest_region_data = hprice_1br_median[hprice_1br_median['Region'] == highest_region].iloc[0, 1:].values

fig2 = go.Figure()
fig2.add_trace(go.Scatter(
    x=quarters,
    y=highest_region_data,
    mode='lines+markers',
    name=highest_region,
    line=dict(color='red', width=2),
    marker=dict(size=5)
))

fig2.update_layout(
    title=f'1BR Flat Median Rent Trend: {highest_region}',
    xaxis_title='Quarter',
    yaxis_title='Median Rent (AUD)',
    hovermode='x unified',
    height=500,
    xaxis=dict(tickangle=-45)
)

fig2.show()

HIGHEST AVERAGE MEDIAN RENT:
Region: Melbourne
Average Median Rent: $354.38


In [29]:
# And the Region with the lowest

lowest_region = region_avg_median.idxmin()
lowest_value = region_avg_median.min()

print(f"LOWEST AVERAGE MEDIAN RENT:")
print(f"Region: {lowest_region}")
print(f"Average Median Rent: ${lowest_value:,.2f}")

# Show trend for this region
lowest_region_data = hprice_1br_median[hprice_1br_median['Region'] == lowest_region].iloc[0, 1:].values

fig3 = go.Figure()
fig3.add_trace(go.Scatter(
    x=quarters,
    y=lowest_region_data,
    mode='lines+markers',
    name=lowest_region,
    line=dict(color='green', width=2),
    marker=dict(size=5)
))

fig3.update_layout(
    title=f'1BR Flat Median Rent Trend: {lowest_region}',
    xaxis_title='Quarter',
    yaxis_title='Median Rent (AUD)',
    hovermode='x unified',
    height=500,
    xaxis=dict(tickangle=-45)
)

fig3.show()

LOWEST AVERAGE MEDIAN RENT:
Region: Towong
Average Median Rent: $97.50


### Observations based on the graphs:
   - Clear upward trend from 1999 to 2025
   - Some interesting fluctuations around 2020-2021, possibly related to COVID.
   - Since Melbourne is known as the city with the biggest number of COVID lockdowns, let's investigate further...

In [ ]:
from scipy import stats
# New library needed to add tendency lines
# Remember to run: pip freeze > requirements.txt

# ENHANCED OVERALL TREND with COVID marker and trend line
median_totals = hprice_1br_median.iloc[:, 1:].sum(axis=0)
quarters = hprice_1br_median.columns[1:].tolist()

trend_df = pd.DataFrame({
    'Quarter': quarters,
    'Total_Median_Rent': median_totals.values
})

# Add a numeric index for trend line calculation
trend_df['Index'] = range(len(trend_df))

# Calculate linear trend line
slope, intercept, r_value, p_value, std_err = stats.linregress(trend_df['Index'], trend_df['Total_Median_Rent'])
trend_df['Trend_Line'] = slope * trend_df['Index'] + intercept

# Find the index of Mar_2020
covid_start_idx = quarters.index('Mar_2020') if 'Mar_2020' in quarters else None

fig1 = go.Figure()

# Actual data
fig1.add_trace(go.Scatter(
    x=trend_df['Quarter'],
    y=trend_df['Total_Median_Rent'],
    mode='lines+markers',
    name='Total Median Rent',
    line=dict(color='royalblue', width=2),
    marker=dict(size=4)
))

# Trend line
fig1.add_trace(go.Scatter(
    x=trend_df['Quarter'],
    y=trend_df['Trend_Line'],
    mode='lines',
    name='Trend Line',
    line=dict(color='orange', width=2, dash='dash')
))

# COVID start marker (March 2020 - first lockdown)
if covid_start_idx is not None:
    fig1.add_vline(
        x=covid_start_idx, 
        line_dash="dash", 
        line_color="red",
        annotation_text="COVID-19<br>First Lockdown<br>(30 Mar 2020)",
        annotation_position="top"
    )

fig1.update_layout(
    title='1 Bedroom Flat: Total Median Rent Trend (1999-2025)',
    xaxis_title='Quarter',
    yaxis_title='Sum of Median Rent (AUD)',
    hovermode='x unified',
    height=600,
    xaxis=dict(tickangle=-45),
    showlegend=True
)

fig1.show()

In [33]:
# ENHANCED MELBOURNE TREND with both lockdown markers
melbourne_data = hprice_1br_median[hprice_1br_median['Region'] == 'Melbourne'].iloc[0, 1:].values

# Find the indices for lockdown dates
covid_start_idx = quarters.index('Mar_2020') if 'Mar_2020' in quarters else None
covid_end_idx = quarters.index('Dec_2021') if 'Dec_2021' in quarters else None

fig2 = go.Figure()

# Melbourne rent data
fig2.add_trace(go.Scatter(
    x=quarters,
    y=melbourne_data,
    mode='lines+markers',
    name='Melbourne Median Rent',
    line=dict(color='red', width=2),
    marker=dict(size=5)
))

# First lockdown (30 March 2020)
if covid_start_idx is not None:
    fig2.add_vline(
        x=covid_start_idx, 
        line_dash="dash", 
        line_color="darkred",
    )

# Final lockdown ended (21 October 2021)
if covid_end_idx is not None:
    fig2.add_vline(
        x=covid_end_idx, 
        line_dash="dash", 
        line_color="green",
    )

# Add shaded area for lockdown period
if covid_start_idx is not None and covid_end_idx is not None:
    fig2.add_vrect(
        x0=covid_start_idx, 
        x1=covid_end_idx,
        fillcolor="gray", 
        opacity=0.2,
        annotation_text="Lockdown Period",
        annotation_position="top"
    )

fig2.update_layout(
    title='1BR Flat Median Rent: Melbourne (with COVID Lockdown Period)',
    xaxis_title='Quarter',
    yaxis_title='Median Rent (AUD)',
    hovermode='x unified',
    height=600,
    xaxis=dict(tickangle=-45),
    showlegend=True
)

fig2.show()

# Calculate the drop and recovery
pre_covid = hprice_1br_median[hprice_1br_median['Region'] == 'Melbourne']['Dec_2019'].values[0]
during_covid_low = hprice_1br_median[hprice_1br_median['Region'] == 'Melbourne']['Jun_2021'].values[0]
post_covid = hprice_1br_median[hprice_1br_median['Region'] == 'Melbourne']['Jun_2025'].values[0]

print(f"MELBOURNE COVID IMPACT ANALYSIS:")
print(f"Pre-COVID (Dec 2019): ${pre_covid:,.0f}")
print(f"Lowest during COVID (Jun 2021): ${during_covid_low:,.0f}")
print(f"Drop: ${pre_covid - during_covid_low:,.0f} ({((during_covid_low - pre_covid) / pre_covid * 100):.1f}%)")
print(f"Current (Jun 2025): ${post_covid:,.0f}")
print(f"Recovery: ${post_covid - during_covid_low:,.0f} ({((post_covid - during_covid_low) / during_covid_low * 100):.1f}%)")

MELBOURNE COVID IMPACT ANALYSIS:
Pre-COVID (Dec 2019): $438
Lowest during COVID (Jun 2021): $310
Drop: $128 (-29.2%)
Current (Jun 2025): $550
Recovery: $240 (77.4%)


### Key Insights from the new version:
   - Linear growth pattern disrupted by COVID-19, especially in Melbourne LGA (a.k.a city), considering during COVID many people left the city centre and went to the suburbs. This was seen in many cities thoughout the world, and is confirmed in Melbourne's case.
   - Market volatility during lockdown period
   - Strong recovery post-2021, after the last lockdown

In [32]:
# Now, let's look at the Count dataset, to observe the patterns in the housing prices
# And see if they match the ones observed for the median prices
# Already adding trend line and COVID lockdown start date

from scipy import stats

# OVERALL TREND with COVID marker and trend line
count_totals = hprice_1br_count.iloc[:, 1:].sum(axis=0)
quarters = hprice_1br_count.columns[1:].tolist()

trend_df = pd.DataFrame({
    'Quarter': quarters,
    'Total_Count_Rent': count_totals.values
})

# Add a numeric index for trend line calculation
trend_df['Index'] = range(len(trend_df))

# Calculate linear trend line
slope, intercept, r_value, p_value, std_err = stats.linregress(trend_df['Index'], trend_df['Total_Count_Rent'])
trend_df['Trend_Line'] = slope * trend_df['Index'] + intercept

# Find the index of Mar_2020
covid_start_idx = quarters.index('Mar_2020') if 'Mar_2020' in quarters else None

fig1 = go.Figure()

# Actual data
fig1.add_trace(go.Scatter(
    x=trend_df['Quarter'],
    y=trend_df['Total_Count_Rent'],
    mode='lines+markers',
    name='Number of Units Available for Rent',
    line=dict(color='royalblue', width=2),
    marker=dict(size=4)
))

# Trend line
fig1.add_trace(go.Scatter(
    x=trend_df['Quarter'],
    y=trend_df['Trend_Line'],
    mode='lines',
    name='Trend Line',
    line=dict(color='orange', width=2, dash='dash')
))

# COVID start marker (March 2020 - first lockdown)
if covid_start_idx is not None:
    fig1.add_vline(
        x=covid_start_idx, 
        line_dash="dash", 
        line_color="red",
        annotation_text="COVID-19<br>First Lockdown<br>(30 Mar 2020)",
        annotation_position="top"
    )

fig1.update_layout(
    title='1 Bedroom Flat: Number of Units Available for Rent Trend (1999-2025)',
    xaxis_title='Quarter',
    yaxis_title='Sum of Units Available',
    hovermode='x unified',
    height=600,
    xaxis=dict(tickangle=-45),
    showlegend=True
)

fig1.show()

# General Numbers

print(f"Number of quarters analyzed: {len(quarters)}")
print(f"Starting total (Jun 1999): {count_totals.iloc[0]:,.0f}")
print(f"Ending total (Jun 2025): {count_totals.iloc[-1]:,.0f}")
print(f"Overall increase: {count_totals.iloc[-1] - count_totals.iloc[0]:,.0f}")

Number of quarters analyzed: 105
Starting total (Jun 1999): 4,332
Ending total (Jun 2025): 7,055
Overall increase: 2,723


### Key Insights
This second general trend graph highlights **the existence of seasonality** in the data (hence, in the rental market). This pattern was already observed in the behaviour of median rent prices, but it's more pronounced when looking at the total number of available units.

As expected, considering that Melbourne is Australia's education capital with many universities, there is significant **fluctuation associated with the academic calendar**. Therefore, the number of available units decreases during vacation periods (June and December), when students return home for holidays. December is also prone to this decrease as it's summer, when people are typically travelling. While this might suggest more units should be available, the opposite occurs, as the rental market tends to hold these units and make them available closer to the beginning of the academic year.

It's also important to note that these numbers reflect the **reality of 1-bedroom flats, which are more affected by the student calendar and relocating young professionals** (singles and couples) - profiles that typically seek this type of housing. Other property types, like larger apartments and houses, might present different behaviour, especially during COVID.

In [43]:
# Calculate sum of units available for rent for each region
region_count_sum = hprice_1br_count.set_index('Region').sum(axis=1, skipna=True)

# Region with the highest availability
highest_region = region_count_sum.idxmax()
highest_value = region_count_sum.max()

print(f"HIGHEST AVAILABILITY FOR RENT - 1BR FLATS:")
print(f"Region: {highest_region}")
print(f"Total Units Available (1999-2025): {highest_value:,.0f}")

# Get units available in Jun 2025 for the highest region
jun_2025_highest = hprice_1br_count.loc[hprice_1br_count['Region'] == highest_region, 'Jun_2025'].values[0]
print(f"Units Available in Jun 2025: {jun_2025_highest:,.0f}")

# Region with the lowest availability
lowest_region = region_count_sum.idxmin()
lowest_value = region_count_sum.min()

print(f"{'='*80}")
print(f"LOWEST AVAILABILITY FOR RENT - 1BR FLATS:")
print(f"Region: {lowest_region}")
print(f"Total Units Available (1999-2025): {lowest_value:,.0f}")

# Get units available in Jun 2025 for the lowest region
jun_2025_lowest = hprice_1br_count.loc[hprice_1br_count['Region'] == lowest_region, 'Jun_2025'].values[0]
print(f"Units Available in Jun 2025: {jun_2025_lowest:,.0f}")

HIGHEST AVAILABILITY FOR RENT - 1BR FLATS:
Region: Melbourne
Total Units Available (1999-2025): 176,155
Units Available in Jun 2025: 2,253
LOWEST AVAILABILITY FOR RENT - 1BR FLATS:
Region: Moyne
Total Units Available (1999-2025): 0
Units Available in Jun 2025: nan


In [44]:
# Creating the graph for Melbourne region, with the highest number of units for rent

melbourne_data = hprice_1br_count[hprice_1br_count['Region'] == 'Melbourne'].iloc[0, 1:].values

# Find the indices for lockdown dates
covid_start_idx = quarters.index('Mar_2020') if 'Mar_2020' in quarters else None
covid_end_idx = quarters.index('Dec_2021') if 'Dec_2021' in quarters else None

fig2 = go.Figure()

# Melbourne rent data
fig2.add_trace(go.Scatter(
    x=quarters,
    y=melbourne_data,
    mode='lines+markers',
    name='Units Available for Rent in Melbourne',
    line=dict(color='red', width=2),
    marker=dict(size=5)
))

# First lockdown (30 March 2020)
if covid_start_idx is not None:
    fig2.add_vline(
        x=covid_start_idx, 
        line_dash="dash", 
        line_color="darkred",
    )

# Final lockdown ended (21 October 2021)
if covid_end_idx is not None:
    fig2.add_vline(
        x=covid_end_idx, 
        line_dash="dash", 
        line_color="green",
    )

# Add shaded area for lockdown period
if covid_start_idx is not None and covid_end_idx is not None:
    fig2.add_vrect(
        x0=covid_start_idx, 
        x1=covid_end_idx,
        fillcolor="gray", 
        opacity=0.2,
        annotation_text="Lockdown Period",
        annotation_position="top"
    )

fig2.update_layout(
    title='1BR Flat Units Available for Rent: Melbourne region',
    xaxis_title='Quarter',
    yaxis_title='Number of Available Units',
    hovermode='x unified',
    height=600,
    xaxis=dict(tickangle=-45),
    showlegend=True
)

fig2.show()

# Calculate the drop and recovery
pre_covid = hprice_1br_count[hprice_1br_count['Region'] == 'Melbourne']['Dec_2019'].values[0]
during_covid_low = hprice_1br_count[hprice_1br_count['Region'] == 'Melbourne']['Jun_2021'].values[0]
post_covid = hprice_1br_count[hprice_1br_count['Region'] == 'Melbourne']['Jun_2025'].values[0]

print(f"MELBOURNE COVID IMPACT ANALYSIS:")
print(f"Pre-COVID (Dec 2019): {pre_covid:,.0f}")
print(f"Lowest during COVID (Jun 2021): {during_covid_low:,.0f}")
print(f"Drop: {pre_covid - during_covid_low:,.0f} ({((during_covid_low - pre_covid) / pre_covid * 100):.1f}%)")
print(f"Current (Jun 2025): {post_covid:,.0f}")
print(f"Recovery: {post_covid - during_covid_low:,.0f} ({((post_covid - during_covid_low) / during_covid_low * 100):.1f}%)")

MELBOURNE COVID IMPACT ANALYSIS:
Pre-COVID (Dec 2019): 1,847
Lowest during COVID (Jun 2021): 3,387
Drop: -1,540 (83.4%)
Current (Jun 2025): 2,253
Recovery: -1,134 (-33.5%)


#### Key Insights
Unlike the previous graph showing median 1BR prices, an increase in available units is observed during the lockdown period. This behaviour is expected, as many people left the city centre and moved to the suburbs. The number of units significantly increased, while prices decreased in an attempt to attract more tenants to the city centre.

Towards the end of COVID lockdowns, the number of available units began decreasing, likely due to the gradual return of university classes and on-site work, with people moving back to the city centre, especially students. Following the end of COVID lockdowns, the availability of 1BR flats returned to its fluctuating pattern, as before, but now with higher overall availability compared to pre-COVID levels.

In [52]:
# Further exploring this, let's take a look at both the availability and the median prices on the same graph 
# for Melbourne area

# Dual-axis plot: Count vs Median for Melbourne
melbourne_count = hprice_1br_count[hprice_1br_count['Region'] == 'Melbourne'].iloc[0, 1:].values
melbourne_median = hprice_1br_median[hprice_1br_median['Region'] == 'Melbourne'].iloc[0, 1:].values

# Find COVID indices
covid_start_idx = quarters.index('Mar_2020') if 'Mar_2020' in quarters else None
covid_end_idx = quarters.index('Dec_2021') if 'Dec_2021' in quarters else None

# Create figure with secondary y-axis
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add Count trace (left y-axis)
fig.add_trace(
    go.Scatter(
        x=quarters,
        y=melbourne_count,
        name="Units Available",
        line=dict(color='blue', width=2),
        mode='lines+markers',
        marker=dict(size=4)
    ),
    secondary_y=False
)

# Add Median trace (right y-axis)
fig.add_trace(
    go.Scatter(
        x=quarters,
        y=melbourne_median,
        name="Median Rent (AUD)",
        line=dict(color='red', width=2),
        mode='lines+markers',
        marker=dict(size=4)
    ),
    secondary_y=True
)

# Add COVID lockdown period shading
if covid_start_idx is not None and covid_end_idx is not None:
    fig.add_vrect(
        x0=covid_start_idx,
        x1=covid_end_idx,
        fillcolor="gray",
        opacity=0.2,
        layer="below",
        line_width=0,
        annotation_text="Lockdown Period",
        annotation_position="top left"
    )
    
    # Add lockdown markers
    fig.add_vline(
        x=covid_start_idx,
        line_dash="dash",
        line_color="darkred"
    )
    
    fig.add_vline(
        x=covid_end_idx,
        line_dash="dash",
        line_color="green"
    )

# Update axes titles
fig.update_xaxes(title_text="Quarter", tickangle=-45)
fig.update_yaxes(title_text="<b>Number of Units Available</b>", secondary_y=False, title_font=dict(color="blue"))
fig.update_yaxes(title_text="<b>Median Rent (AUD)</b>", secondary_y=True, title_font=dict(color="red"))

# Update layout
fig.update_layout(
    title="Melbourne 1BR Flats: Supply vs Price (1999-2025)",
    hovermode='x unified',
    height=600,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()

# Print key insights
print(f"\nPRE-COVID (Dec 2019):")
pre_covid_count = hprice_1br_count[hprice_1br_count['Region'] == 'Melbourne']['Dec_2019'].values[0]
pre_covid_median = hprice_1br_median[hprice_1br_median['Region'] == 'Melbourne']['Dec_2019'].values[0]
print(f"  Units Available: {pre_covid_count:,.0f}")
print(f"  Median Rent: ${pre_covid_median:,.0f}")

print(f"\nPEAK COVID IMPACT (Jun 2021):")
peak_covid_count = hprice_1br_count[hprice_1br_count['Region'] == 'Melbourne']['Jun_2021'].values[0]
peak_covid_median = hprice_1br_median[hprice_1br_median['Region'] == 'Melbourne']['Jun_2021'].values[0]
print(f"  Units Available: {peak_covid_count:,.0f} ({((peak_covid_count - pre_covid_count) / pre_covid_count * 100):+.1f}%)")
print(f"  Median Rent: ${peak_covid_median:,.0f} ({((peak_covid_median - pre_covid_median) / pre_covid_median * 100):+.1f}%)")

print(f"\nCURRENT (Jun 2025):")
current_count = hprice_1br_count[hprice_1br_count['Region'] == 'Melbourne']['Jun_2025'].values[0]
current_median = hprice_1br_median[hprice_1br_median['Region'] == 'Melbourne']['Jun_2025'].values[0]
print(f"  Units Available: {current_count:,.0f} ({((current_count - pre_covid_count) / pre_covid_count * 100):+.1f}% vs pre-COVID)")
print(f"  Median Rent: ${current_median:,.0f} ({((current_median - pre_covid_median) / pre_covid_median * 100):+.1f}% vs pre-COVID)")

print(f"\n{'='*80}")
print(f"KEY INSIGHT:")
print(f"Supply INCREASED during COVID {((peak_covid_count - pre_covid_count) / pre_covid_count * 100):.1f}% but prices FELL {((peak_covid_median - pre_covid_median) / pre_covid_median * 100):.1f}%")
print(f"Now: Supply went DOWN comparing to COVID {((current_count - pre_covid_count) / pre_covid_count * 100):.1f}% but prices UP {((current_median - pre_covid_median) / pre_covid_median * 100):.1f}%")


PRE-COVID (Dec 2019):
  Units Available: 1,847
  Median Rent: $438

PEAK COVID IMPACT (Jun 2021):
  Units Available: 3,387 (+83.4%)
  Median Rent: $310 (-29.2%)

CURRENT (Jun 2025):
  Units Available: 2,253 (+22.0% vs pre-COVID)
  Median Rent: $550 (+25.6% vs pre-COVID)

KEY INSIGHT:
Supply INCREASED during COVID 83.4% but prices FELL -29.2%
Now: Supply went DOWN comparing to COVID 22.0% but prices UP 25.6%


---

## 📊 Summary & Key Findings

### What We Did in This Notebook

This analysis explored **26 years of rental market data** (1999-2025) for 1-bedroom flats across Victoria, Australia, with special focus on Melbourne's market dynamics.

#### Data Processing:
- ✅ Cleaned and restructured government Excel data (multiple sheets)
- ✅ Separated Count (availability) and Median (price) datasets
- ✅ Handled missing values and data quality issues
- ✅ Created reproducible CSV files for further analysis

---

### 🔑 Key Findings

#### 1. **Overall Market Growth**
- **Median rent increase:** $180 (Jun 1999) → $565 (Jun 2025) = **+214%**
- **Steady upward trend** with clear linear growth pattern
- **Seasonality present:** December consistently shows lower activity

#### 2. **COVID-19 outcome in Melbourne Rental Market**

**Pre-COVID (Dec 2019):**
- Available units: 1,847
- Median rent: $450

**Peak COVID Impact (Jun 2021):**
- Available units: 3,387 (**+83%** ⬆️)
- Median rent: $310 (**-31%** ⬇️)

**Current State (Jun 2025):**
- Available units: 2,253 (+22% vs pre-COVID)
- Median rent: $565 (**+26%** vs pre-COVID)

**The Story:** During lockdowns, supply exploded as people fled the city, causing prices to crash. Post-lockdown, demand returned strongly but supply remained constrained, driving prices to record highs.

#### 3. **Seasonality Patterns**
- **Strong seasonal fluctuations** in availability (more pronounced than prices)
- **December dips:** Lower availability due to summer holidays and market slowdown
- **March peaks:** Aligned with academic year start and post-holiday relocations
- **Pattern amplified** for student-oriented properties (1BR flats)

#### 4. **Regional Variations**
- **Highest median rent:** Melbourne (education and employment hub)
- **Lowest median rent:** Towong (rural/regional area)
- **Highest availability:** Melbourne (largest market)
- **Lowest availability:** Moyne (small regional council)

---

### 💡 Insights & Implications

1. **Supply-Demand Economics in Action:** The inverse relationship between availability and price is textbook economics - vividly demonstrated during COVID.

2. **Student Market Influence:** Strong seasonality suggests 1BR flat market is heavily influenced by university calendars and young professional relocations.

3. **Urban Impact:** COVID-19 triggered significant urban-to-suburban migration, with lasting effects on both supply and pricing.

4. **Market Recovery:** Post-pandemic recovery has been asymmetric - prices recovered faster than supply normalized, creating affordability challenges.

---

### 🚀 Next Steps

#### 📍 **Notebook 2: Geospatial Analysis** (1BR Focus)
- Map rental prices across Melbourne regions
- Identify spatial patterns and hotspots
- Analyze urban vs suburban trends
- Understand regional variations

#### 📈 **Notebook 3: Time Series Forecasting** (1BR Focus)
- Apply SARIMA models to account for seasonality
- Forecast future rental prices
- Predict market trends for 2026-2027

#### 🤖 **Notebook 4: Predictive Modeling** (Multi-property)
- Build models incorporating property type (1BR, 2BR, 3BR, Houses)
- Feature engineering with spatial and temporal data
- Model evaluation and deployment

#### 🏘️ **Future Extension: Comparative Property Analysis**
- Expand cleaning pipeline to 2BR, 3BR flats and houses
- Compare market dynamics across property types
- Analyze different demographics (students vs families)
- Compare with other australian cities

---

### 📁 Outputs from This Notebook

**Processed Data:**
- `data/processed/1br_flat_median.csv` - Clean median rent data
- `data/processed/1br_flat_count.csv` - Clean availability data

**Key Visualizations:**
- Overall trend with COVID marker
- Melbourne supply vs price dual-axis chart
- Regional comparisons

---

### 🛠️ Tools Used
- **Python 3.x**
- **Pandas** - Data manipulation
- **Plotly** - Interactive visualizations
- **SciPy** - Statistical analysis
- **Jupyter Notebook** - Analysis environment

---

**Analysis Date:** January 2026  
**Data Source:** Victorian Government - Quarterly Median Rents  
**Geographic Focus:** Victoria, Australia (emphasis on Melbourne)

---

*Ready to explore the spatial dimension? Let's map it! See you on Notebook 2🗺️*